# ADS-508-01-SP23 Team 8: Final Project

# Split and Preprocess ABT

Much of the code is modified from `Fregly, C., & Barth, A. (2021). Data science on AWS: Implementing end-to-end, continuous AI and machine learning pipelines. O’Reilly.`

## Install missing dependencies

[PyAthena](https://pypi.org/project/PyAthena/) is a Python DB API 2.0 (PEP 249) compliant client for Amazon Athena.

In [1]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0
!pip install missingno

## Globally import libraries

In [2]:
import boto3
from botocore.client import ClientError
import sagemaker
import pandas as pd
from pyathena import connect
from IPython.core.display import display, HTML
import missingno as msno
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import VarianceThreshold
import datetime as dt

## Instantiate AWS SageMaker and S3 sessions

In [3]:
session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
def_bucket = sagemaker_session.default_bucket()
bucket = 'sagemaker-us-east-ads508-sp23-t8'

s3 = boto3.Session().client(service_name="s3",
                            region_name=region)

In [4]:
setup_s3_bucket_passed = False
ingest_create_athena_db_passed = False
ingest_create_athena_table_tsv_passed = False

In [5]:
print(f"Default bucket: {def_bucket}")
print(f"Public T8 bucket: {bucket}")

Default bucket: sagemaker-us-east-1-657724983756
Public T8 bucket: sagemaker-us-east-ads508-sp23-t8


## Verify S3 Bucket Creation

In [6]:
%%bash

aws s3 ls s3://${bucket}/

2023-03-16 17:05:02 aws-athena-query-results-657724983756-us-east-1
2023-03-02 16:56:48 sagemaker-studio-657724983756-5nh7ydsouq7
2023-03-02 17:25:41 sagemaker-studio-657724983756-7yc8bp8xk0b
2023-03-02 17:01:51 sagemaker-us-east-1-657724983756
2023-03-17 05:19:31 sagemaker-us-east-ads508-sp23-t8


In [7]:
response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print(f"[ERROR] Cannot find bucket {bucket} in {response} due to {e}.")

{'ResponseMetadata': {'RequestId': '5V65S587SZ884CAC', 'HostId': 'kqS9PmFbC6kZtQU49rbdqUS3GIGtN/8770YHedDm0fWYLFCd+KE5vNRhgStW0UWaaEFtHIO4ToU=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'kqS9PmFbC6kZtQU49rbdqUS3GIGtN/8770YHedDm0fWYLFCd+KE5vNRhgStW0UWaaEFtHIO4ToU=', 'x-amz-request-id': '5V65S587SZ884CAC', 'date': 'Mon, 27 Mar 2023 18:55:46 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


In [8]:
%store setup_s3_bucket_passed

Stored 'setup_s3_bucket_passed' (bool)


## Pass in ABT from CSV

In [9]:
s3_abt_csv_path = f"s3://{def_bucket}/team_8_data/abt/abt_encoded_df01.csv"
abt_encoded_df01 = pd.read_csv(s3_abt_csv_path)

### Perform train/test split

In [10]:
y01 = ['childpoverty']
abt_encoded_y01_vc01 = abt_encoded_df01[y01].to_numpy()
print(abt_encoded_y01_vc01.shape)
display(abt_encoded_y01_vc01[0:11])
abt_encoded_x01_df01 = abt_encoded_df01.drop(y01, axis=1)
print(abt_encoded_x01_df01.shape)
display(abt_encoded_x01_df01.head(11))

abt_encoded_x01_df01['boroughs'] = abt_encoded_x01_df01['borough_bronx'].astype(int).astype(str) + abt_encoded_x01_df01['borough_brooklyn'].astype(int).astype(str) + abt_encoded_x01_df01['borough_manhattan'].astype(int).astype(str) + abt_encoded_x01_df01['borough_queens'].astype(int).astype(str) + abt_encoded_x01_df01['borough_staten island'].astype(int).astype(str)
display(abt_encoded_x01_df01.head(5))
train_x01, test_x01, train_y01, test_y01 = train_test_split(abt_encoded_x01_df01,
                                                            abt_encoded_y01_vc01,
                                                            test_size=.2,
                                                            stratify=abt_encoded_x01_df01[['boroughs']],
                                                            shuffle=True,
                                                            random_state=1699)

train_x01 = train_x01.drop(['boroughs'], axis=1)
test_x01 = test_x01.drop(['boroughs'], axis=1)

print(f'{train_x01.shape}')
print(f'{train_y01.shape}')
print(f'\n{test_x01.shape}')
print(f'{test_y01.shape}')

(31605, 1)


array([[20.7],
       [23.6],
       [35.9],
       [31.5],
       [67.7],
       [68.3],
       [ 0. ],
       [62.4],
       [64.9],
       [63.1],
       [ 6.5]])

(31605, 49)


,borough_bronx,borough_brooklyn,borough_manhattan,borough_queens,borough_staten island,relative_data_year_-4,relative_data_year_-3,relative_data_year_-2,relative_data_year_-1,relative_data_year_0,...,walk,othertransp,workathome,meancommute,employed,privatework,publicwork,selfemployed,familywork,unemployment
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.9,0.0,0.0,43.0,2308.0,80.8,16.2,2.9,0.0,7.7
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.4,0.5,2.1,45.0,2675.0,71.7,25.3,2.5,0.6,9.5
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,8.6,1.6,1.7,38.8,2120.0,75.0,21.3,3.8,0.0,8.7
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,3.0,2.4,6.2,45.4,1083.0,76.8,15.5,7.7,0.0,19.2
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,4.3,1.0,0.0,46.0,2508.0,71.0,21.3,7.7,0.0,17.2
5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,14.0,1.5,4.1,42.7,1191.0,74.2,16.1,9.7,0.0,18.9
6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,113.0,62.8,37.2,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,17.7,1.8,2.7,35.5,1691.0,85.1,8.3,6.1,0.5,9.4
8,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,18.0,0.0,1.6,42.8,1102.0,86.9,8.5,4.5,0.0,15.2
9,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,7.1,0.7,0.5,44.0,1559.0,75.0,14.0,11.0,0.0,10.6


,borough_bronx,borough_brooklyn,borough_manhattan,borough_queens,borough_staten island,relative_data_year_-4,relative_data_year_-3,relative_data_year_-2,relative_data_year_-1,relative_data_year_0,...,othertransp,workathome,meancommute,employed,privatework,publicwork,selfemployed,familywork,unemployment,boroughs
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,43.0,2308.0,80.8,16.2,2.9,0.0,7.7,10000
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.5,2.1,45.0,2675.0,71.7,25.3,2.5,0.6,9.5,10000
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.6,1.7,38.8,2120.0,75.0,21.3,3.8,0.0,8.7,10000
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.4,6.2,45.4,1083.0,76.8,15.5,7.7,0.0,19.2,10000
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,46.0,2508.0,71.0,21.3,7.7,0.0,17.2,10000


(25284, 49)
(25284, 1)

(6321, 49)
(6321, 1)


#### Examine featues with near zero variances

In [11]:
# Review near-zero variance (NZV) features for possible removal
train_x01_nzv_fit = VarianceThreshold(.025).fit(train_x01)
train_x01_nzv_vc01 = train_x01_nzv_fit.transform(train_x01)
#print(train_x01_nzv_vc01)
#print(train_x01_nzv_vc01.shape)

# Get the names of the selected features
train_x01_nzv_fit_select_features = train_x01.columns[train_x01_nzv_fit.get_support()]

train_x01_nzv_df01 = pd.DataFrame(train_x01_nzv_vc01,
                                  columns=train_x01_nzv_fit_select_features)

display(train_x01_nzv_df01.head(5))
print(f'NZV transformed matrix dimensions = {train_x01_nzv_df01.shape}')

print(f'\n{train_x01.shape[1] - train_x01_nzv_df01.shape[1]} near zero variance features were eliminated')

print(train_x01.columns)
print(train_x01_nzv_df01.columns)

,borough_bronx,borough_brooklyn,borough_manhattan,borough_queens,borough_staten island,relative_data_year_-4,relative_data_year_-3,relative_data_year_-2,relative_data_year_-1,relative_data_year_0,...,walk,othertransp,workathome,meancommute,employed,privatework,publicwork,selfemployed,familywork,unemployment
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,4.6,0.6,3.9,42.9,3293.0,81.3,13.2,5.6,0.0,3.4
1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,7.2,2.4,3.4,30.4,3872.0,86.0,5.9,8.1,0.0,6.5
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3.7,0.4,3.1,43.0,2014.0,78.6,13.5,7.9,0.0,6.7
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,4.1,7.0,6.8,40.4,1782.0,82.7,9.2,8.1,0.0,7.8
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.6,1.2,0.8,42.0,801.0,76.3,20.1,3.6,0.0,8.9


NZV transformed matrix dimensions = (25284, 49)

0 near zero variance features were eliminated
Index(['borough_bronx', 'borough_brooklyn', 'borough_manhattan',
       'borough_queens', 'borough_staten island', 'relative_data_year_-4',
       'relative_data_year_-3', 'relative_data_year_-2',
       'relative_data_year_-1', 'relative_data_year_0',
       'complaint_type_FELONY', 'complaint_type_MISDEMEANOR',
       'complaint_type_VIOLATION', 'annual_evictions_x_borough',
       'annual_complaint_counts', 'annual_grad_n', 'annual_dropped_out_n',
       'totalpop', 'men', 'women', 'hispanic', 'white', 'black', 'native',
       'asian', 'citizen', 'income', 'incomeerr', 'incomepercap',
       'incomepercaperr', 'poverty', 'professional', 'service', 'office',
       'construction', 'production', 'drive', 'carpool', 'transit', 'walk',
       'othertransp', 'workathome', 'meancommute', 'employed', 'privatework',
       'publicwork', 'selfemployed', 'familywork', 'unemployment'],
      dtype='

## Release Resources

In [12]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [13]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>